In [3]:
'''
Please note that the shape_predictor_68_face_landmarks.dat file, which contains the pretrained model 
for detecting the facial landmarks, is not included in the dlib library
, and you will have to download it separately. 
You can find it here: http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2

Unzip it and provide the correct path in the script.
'''

import time
import dlib
import cv2
import numpy as np
from scipy.spatial import distance as dist

def get_ear(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

# Indices of left and right eyes in dlib's facial landmarks
LEFT_EYE_INDICES = list(range(36, 42))
RIGHT_EYE_INDICES = list(range(42, 48))

EAR_THRESHOLD = 0.28  # Eye Aspect Ratio threshold for blink detection
PAUSE_TIME_THRESHOLD = 2.0  # Pause time threshold (in seconds)

cap = cv2.VideoCapture(0)
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("../large_files/shape_predictor_68_face_landmarks.dat")

blink_count = 0
blink_count_sequence = []
prev_blink_detected = False
blink_start_time = None

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    rects = detector(gray, 0)

    for rect in rects:
        shape = predictor(gray, rect)
        shape = np.array([[p.x, p.y] for p in shape.parts()])
        left_eye = shape[LEFT_EYE_INDICES]
        right_eye = shape[RIGHT_EYE_INDICES]
        left_ear = get_ear(left_eye)
        right_ear = get_ear(right_eye)
        avg_ear = (left_ear + right_ear) / 2.0

        cv2.putText(frame, f"EAR: {avg_ear:.2f}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(frame, f"Blinks: {blink_count}", (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        blink_detected = avg_ear < EAR_THRESHOLD

        if blink_detected and not prev_blink_detected:  # Blink start
            blink_start_time = time.time()

        if not blink_detected and prev_blink_detected:  # Blink end
            blink_end_time = time.time()
            blink_duration = blink_end_time - blink_start_time

            if blink_duration >= PAUSE_TIME_THRESHOLD:
                blink_count_sequence.append(blink_count)
                blink_count = 0
            else:
                blink_count += 1

        prev_blink_detected = blink_detected

    cv2.imshow("Frame", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Append remaining blink count to sequence
if blink_count > 0:
    blink_count_sequence.append(blink_count)
AUTHENTICATION_PATTERN = [3, 2]  # Pattern of consecutive blinks for authentication
# Check if blink sequence matches authentication pattern
if blink_count_sequence == AUTHENTICATION_PATTERN:
    print("Authenticated!")
print("Blink sequence:", blink_count_sequence)
cap.release()
cv2.destroyAllWindows()


Authenticated!
Blink sequence: [3, 2]


In [56]:
import time
import dlib
import cv2
import numpy as np
from scipy.spatial import distance as dist

def get_ear(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

# Indices of left and right eyes in dlib's facial landmarks
LEFT_EYE_INDICES = list(range(36, 42))
RIGHT_EYE_INDICES = list(range(42, 48))

EAR_THRESHOLD = 0.236  # Eye Aspect Ratio threshold for blink detection

cap = cv2.VideoCapture(0)
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

left_blink_count = 0
right_blink_count = 0
prev_left_blink_detected = False
prev_right_blink_detected = False

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    rects = detector(gray, 0)

    for rect in rects:
        shape = predictor(gray, rect)
        shape = np.array([[p.x, p.y] for p in shape.parts()])
        left_eye = shape[LEFT_EYE_INDICES]
        right_eye = shape[RIGHT_EYE_INDICES]
        left_ear = get_ear(left_eye)
        right_ear = get_ear(right_eye)

        left_blink_detected = left_ear < EAR_THRESHOLD
        right_blink_detected = right_ear < EAR_THRESHOLD

        if left_blink_detected and not prev_left_blink_detected:  # Left eye blink start
            left_blink_count += 1
        if right_blink_detected and not prev_right_blink_detected:  # Right eye blink start
            right_blink_count += 1

        prev_left_blink_detected = left_blink_detected
        prev_right_blink_detected = right_blink_detected

        cv2.putText(frame, f"Left Blinks: {left_blink_count}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(frame, f"Right Blinks: {right_blink_count}", (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    cv2.imshow("Frame", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 

In [41]:
import numpy as np
import cv2
import dlib
import time
from scipy.spatial import distance as dist

def get_ear(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

def midpoint(p1, p2):
    return int((p1.x + p2.x)/2), int((p1.y + p2.y)/2)

# Indices of left and right eyes in dlib's facial landmarks
LEFT_EYE_INDICES = list(range(36, 42))
RIGHT_EYE_INDICES = list(range(42, 48))

EAR_THRESHOLD = 0.2  # Eye Aspect Ratio threshold for blink detection
PAUSE_TIME_THRESHOLD = 2.0  # Pause time threshold (in seconds)
EYE_MOVEMENT_THRESHOLD = 8.5  # Threshold for detecting rapid eye movement

cap = cv2.VideoCapture(0)
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

blink_count = 0
blink_count_sequence = []
prev_blink_detected = False
blink_start_time = None
prev_eye_position = None

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    rects = detector(gray, 0)

    for rect in rects:
        shape = predictor(gray, rect)
        shape = np.array([[p.x, p.y] for p in shape.parts()])
        left_eye = shape[LEFT_EYE_INDICES]
        right_eye = shape[RIGHT_EYE_INDICES]
        left_ear = get_ear(left_eye)
        right_ear = get_ear(right_eye)
        avg_ear = (left_ear + right_ear) / 2.0

        # Calculate the vertical position of the eyes
        eye_position = (left_eye[3, 1] + right_eye[3, 1]) / 2.0

        cv2.putText(frame, f"EAR: {avg_ear:.2f}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(frame, f"Blinks: {blink_count}", (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        
        blink_detected = avg_ear < EAR_THRESHOLD

        if prev_eye_position is not None and abs(eye_position - prev_eye_position) > EYE_MOVEMENT_THRESHOLD:
            blink_detected = False  # Disregard the blink if the eye position has changed too much

        if blink_detected and not prev_blink_detected:  # Blink start
            blink_start_time = time.time()

        if not blink_detected and prev_blink_detected:  # Blink end
            blink_end_time = time.time()
            blink_duration = blink_end_time - blink_start_time

            if blink_duration >= PAUSE_TIME_THRESHOLD:
                blink_count_sequence.append(blink_count)
                blink_count = 0
            blink_count += 1

        prev_blink_detected = blink_detected
        prev_eye_position = eye_position

    cv2.imshow("Frame", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

print("Blink sequence:", blink_count_sequence)
cap.release()
cv2.destroyAllWindows()


Blink sequence: [14, 2, 6, 4]
